In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import torchvision

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import save_image

import pandas as pd
import numpy as np
import scipy.io
import skimage.io

from PIL import Image, ImageFilter

# Define Model

In [2]:
class Encoder(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(out_channel)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x)
        x, idx = F.max_pool2d(x, kernel_size=2, stride=2, return_indices=True)
        return x, idx

In [3]:
class Decoder(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(out_channel)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x)
        return x

In [4]:
class SegNetBasic(nn.Module):
    """ 
        SegNet Basic is a smaller version of SegNet
        Please refer to this repository:
        https://github.com/0bserver07/Keras-SegNet-Basic
    """
    
    def __init__(self, in_channel, out_channel):
        super().__init__()
        
        self.encoder1 = Encoder(in_channel, 64)
        self.encoder2 = Encoder(64, 80)
        self.encoder3 = Encoder(80, 96)
        self.encoder4 = Encoder(96, 128)
        
        self.decoder1 = Decoder(128, 96)
        self.decoder2 = Decoder(96, 80)
        self.decoder3 = Decoder(80, 64)
        self.decoder4 = Decoder(64, out_channel)
        
    def forward(self, x):
        size1 = x.size()
        x, idx1 = self.encoder1(x)

        size2 = x.size()
        x, idx2 = self.encoder2(x)

        size3 = x.size()
        x, idx3 = self.encoder3(x)
        
        size4 = x.size()
        x, idx4 = self.encoder4(x)

        x = F.max_unpool2d(x, idx4, kernel_size=2, stride=2, output_size=size4)
        x = self.decoder1(x)
        
        x = F.max_unpool2d(x, idx3, kernel_size=2, stride=2, output_size=size3)
        x = self.decoder2(x)

        x = F.max_unpool2d(x, idx2, kernel_size=2, stride=2, output_size=size2)
        x = self.decoder3(x)

        x = F.max_unpool2d(x, idx1, kernel_size=2, stride=2, output_size=size1)
        x = self.decoder4(x)

        return x

# Dataset

In [7]:
# count the number of pixels in each affordance
import glob

path_list = glob.glob('./part-affordance-dataset/tools/*', recursive=True)
print(path_list)
print(len(path_list))

['./part-affordance-dataset/tools/mug_15', './part-affordance-dataset/tools/knife_12', './part-affordance-dataset/tools/spoon_01', './part-affordance-dataset/tools/ladle_04', './part-affordance-dataset/tools/trowel_02', './part-affordance-dataset/tools/scissors_01', './part-affordance-dataset/tools/bowl_08', './part-affordance-dataset/tools/tenderizer_02', './part-affordance-dataset/tools/shovel_02', './part-affordance-dataset/tools/bowl_07', './part-affordance-dataset/tools/spoon_02', './part-affordance-dataset/tools/saw_03', './part-affordance-dataset/tools/cup_04', './part-affordance-dataset/tools/spoon_06', './part-affordance-dataset/tools/spoon_04', './part-affordance-dataset/tools/spoon_08', './part-affordance-dataset/tools/mug_19', './part-affordance-dataset/tools/pot_01', './part-affordance-dataset/tools/knife_11', './part-affordance-dataset/tools/scoop_02', './part-affordance-dataset/tools/bowl_02', './part-affordance-dataset/tools/cup_05', './part-affordance-dataset/tools/mug

In [8]:
105*50

5250

In [9]:
image_path_list = []

for path in path_list:
    i = glob.glob(path + '/*rgb.jpg')
    image_path_list += i

SyntaxError: invalid syntax (<ipython-input-9-dc0bd215e457>, line 5)

In [10]:
class_path_list = []

for path in image_path_list:
    c = path[:-7] + 'label.mat'
    class_path_list.append(c)

NameError: name 'image_path_list' is not defined

In [ ]:
df = pd.DataFrame({
    'image_path': image_path_list,
    'class_path': class_path_list
})

In [ ]:
# # write data as csv_file
# df.to_csv('image_class_path.csv', index=None)

In [ ]:
df = pd.read_csv('image_class_path.csv')

In [ ]:
df_s = df.sample(frac=1, random_state=2)

In [9]:
df_s

,image_path,class_path
6650,./part-affordance-dataset/tools/bowl_04/bowl_0...,./part-affordance-dataset/tools/bowl_04/bowl_0...
16785,./part-affordance-dataset/tools/scissors_06/sc...,./part-affordance-dataset/tools/scissors_06/sc...
21334,./part-affordance-dataset/tools/knife_06/knife...,./part-affordance-dataset/tools/knife_06/knife...
10676,./part-affordance-dataset/tools/turner_08/turn...,./part-affordance-dataset/tools/turner_08/turn...
3308,./part-affordance-dataset/tools/saw_03/saw_03_...,./part-affordance-dataset/tools/saw_03/saw_03_...
1304,./part-affordance-dataset/tools/trowel_02/trow...,./part-affordance-dataset/tools/trowel_02/trow...
23177,./part-affordance-dataset/tools/turner_02/turn...,./part-affordance-dataset/tools/turner_02/turn...
28795,./part-affordance-dataset/tools/scissors_04/sc...,./part-affordance-dataset/tools/scissors_04/sc...
4522,./part-affordance-dataset/tools/spoon_08/spoon...,./part-affordance-dataset/tools/spoon_08/spoon...
5254,./part-affordance-dataset/tools/knife_11/knife...,./part-affordance-dataset/tools/knife_11/knife...


In [20]:
len(image_path_list)

28843

In [21]:
28843*0.8

23074.4

In [10]:
df_train = df_s[:23100]

In [11]:
df_train.to_csv('train.csv', index=None)

In [12]:
df_test = df_s[23100:]

In [13]:
df_test.to_csv('test.csv', index=None)

### the number of the pixels before center crop

In [172]:
cnt_dict = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0}

for path in class_path_list:
    mat = scipy.io.loadmat(path)
    num, cnt = np.unique(mat['gt_label'], return_counts=True)
    
    for n, c in zip(num, cnt):
        cnt_dict[n] += c

cnt_dict

0: 8723340880,<br>
 1: 34159284,<br>
 2: 16009552,<br>
 3: 12433420,<br>
 4: 38476964,<br>
 5: 6773540,<br>
 6: 9273880,<br>
 7: 20102080<br>

# Define Dataset Class

In [5]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import pandas as pd
import numpy as np
import scipy.io
import skimage.io
from PIL import Image, ImageFilter
from sklearn.model_selection import train_test_split

In [11]:
class PartAffordanceDataset(Dataset):
    """Part Affordance Dataset"""
    
    def __init__(self, csv_file, transform=None):
        super().__init__()
        
        self.image_class_path = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.image_class_path)
    
    def __getitem__(self, idx):
        image_path = self.image_class_path.iloc[idx, 0]
        class_path = self.image_class_path.iloc[idx, 1]
        image = skimage.io.imread(image_path) # read as numpy array
        cls = scipy.io.loadmat(class_path)["gt_label"]
        
        sample = {'image': image, 'class': cls}
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [12]:
def crop_center_numpy(array, crop_height, crop_weight):
    h, w = array.shape
    return array[h//2 - crop_height//2: h//2 + crop_height//2,
                 w//2 - crop_weight//2: w//2 + crop_weight//2
                ]

In [13]:
def crop_center_pil_image(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

In [14]:
class CenterCrop(object):
    def __call__(self, sample):
        image, cls = sample['image'], sample['class']
        
        image = Image.fromarray(np.uint8(image))
        
        image = crop_center_pil_image(image, 320, 240)
        cls = crop_center_numpy(cls, 240, 320)
        
        image = np.asarray(image)
        
        return {'image': image, 'class': cls}

In [15]:
class ToTensor(object):
    def __call__(self, sample):
        image, cls = sample['image'], sample['class']
        
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image).float(), 
                'class': torch.from_numpy(cls).long()}

In [16]:
# data = PartAffordanceDataset('image_class_path.csv',
#                                 transform=transforms.Compose([
#                                     CenterCrop(),
#                                     ToTensor()
#                                 ]))

# data_loader = DataLoader(data, batch_size=10, shuffle=False)

In [17]:
# mean = 0
# std = 0
# n = 0

# for sample in data_loader:
#     img = sample['image']   
#     img = img.view(len(img), 3, -1)
#     mean += img.mean(2).sum(0)
#     std += img.std(2).sum(0)
#     n += len(img)
    
# mean /= n
# std /= n

In [18]:
mean=[55.8630, 59.9099, 91.7419]
std=[31.6852, 29.8496, 19.0835]

In [19]:
class Normalize(object):
    def __call__(self, sample):
        image, cls = sample['image'], sample['class']
        
        image = transforms.functional.normalize(image, mean, std)
        
        return {'image': image, 'class': cls}

In [15]:
train_data = PartAffordanceDataset('train.csv',
                                transform=transforms.Compose([
                                    CenterCrop(),
                                    ToTensor(),
                                    Normalize()
                                ]))

In [16]:
test_data = PartAffordanceDataset('test.csv',
                                transform=transforms.Compose([
                                    CenterCrop(),
                                    ToTensor(),
                                    Normalize()
                                ]))

In [17]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

### count the number of pixels in each class after center crop

In [25]:
dataset = PartAffordanceDataset('image_class_path.csv',
                                transform=transforms.Compose([
                                    CenterCrop(),
                                    ToTensor()
                                ]))
data_laoder = DataLoader(dataset, batch_size=100, shuffle=False)

In [175]:
cnt_dict = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0}

for sample in data_laoder:
    img = sample['class'].numpy()
    
    num, cnt = np.unique(img, return_counts=True)
    
    for n, c in zip(num, cnt):
        cnt_dict[n] += c

cnt_dict

0: 2078085712,  
 1: 34078992,  
 2: 15921090,  
 3: 12433420,  
 4: 38473752,  
 5: 6773528,  
 6: 9273826,  
 7: 20102080  

# Training

In [18]:
from tensorboardX import SummaryWriter
import tqdm

In [19]:
def eval_model(model, test_loader, device='cpu'):
    model.eval()
    
    intersection = torch.zeros(8)   # the dataset has 8 classes including background
    union = torch.zeros(8)
    
    for sample in test_loader:
        x, y = sample['image'], sample['class']
        
        x = x.to(device)
        y = y.to(device)
        
        with torch.no_grad():
            _, ypred = model(x).max(1)    # y_pred.shape => (N, 240, 320)
        
        for i in range(8):
            y_i = (y == i)           
            ypred_i = (ypred == i)   
            
            inter = (y_i.byte() & ypred_i.byte()).float().sum().to('cpu')
            intersection[i] += inter
            union[i] += (y_i.float().sum() + ypred_i.float().sum()).to('cpu') - inter
    
    """ iou[i] is the IoU of class i """
    iou = intersection / union
    
    return iou

In [20]:
def train_model(model, train_loader, test_loader, optimizer_cls=optim.Adam, 
                criterion=nn.CrossEntropyLoss(), max_epoch=200, device='cpu', writer=None):
    
    model.to(device)
    
    train_losses = []
    val_iou = []
    mean_iou = []
    best_iou = 0.0
    
    optimizer = optimizer_cls(model.parameters(), lr=0.01)
    
    for epoch in range(max_epoch):
        model.train()
        running_loss = 0.0
        
        for i, sample in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
            optimizer.zero_grad()
            
            x, y = sample['image'], sample['class']
            
            x = x.to(device)
            y = y.to(device)

            h = model(x)
            loss = criterion(h, y)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        train_losses.append(running_loss / i)
        
        val_iou.append(eval_model(model, test_loader, device))
        mean_iou.append(val_iou[-1].mean().item())
        
        if best_iou < mean_iou[-1]:
            best_iou = mean_iou[-1]
            torch.save(model.state_dict(), "./SegNet_without_class_weights_results/best_iou_model.prm")
        
        if writer is not None:
            writer.add_scalar("train_loss", train_losses[-1], epoch)
            writer.add_scalar("mean_IoU", mean_iou[-1], epoch)
            writer.add_scalars("class_IoU", {'iou of class 0': val_iou[-1][0],
                                           'iou of class 1': val_iou[-1][1],
                                           'iou of class 2': val_iou[-1][2],
                                           'iou of class 3': val_iou[-1][3],
                                           'iou of class 4': val_iou[-1][4],
                                           'iou of class 5': val_iou[-1][5],
                                           'iou of class 6': val_iou[-1][6],
                                           'iou of class 7': val_iou[-1][7]}, epoch)
            
        print(epoch, train_losses[-1], mean_iou[-1])
        
    torch.save(model.state_dict(), "./SegNet_without_class_weights_results/final_model.prm")

In [21]:
# num0 = 2078085712
# ratio = [1, num0 / 34078992, num0 / 15921090, num0/12433420, 
#          num0 / 38473752, num0 / 6773528, num0 / 9273826, num0 / 20102080]
# class_weight = torch.tensor(ratio, dtype=torch.float32) / 100

In [22]:
model = SegNetBasic(3, 8)
writer = SummaryWriter("./SegNet_without_class_weights_results/")
train_model(model, train_loader, test_loader, device="cuda", writer=writer)

  0%|          | 0/2360 [00:00<?, ?it/s]

0 0.25749609359761527 0.34269246459007263


  0%|          | 0/2360 [00:00<?, ?it/s]

1 0.07439514800460852 0.602653443813324


  0%|          | 1/2360 [00:00<06:05,  6.45it/s]

...

197 0.012849311990013575 0.873960018157959


  0%|          | 1/2360 [00:00<05:26,  7.24it/s]

198 0.012817848483065003 0.8674715757369995


100%|██████████| 2360/2360 [07:05<00:00,  5.55it/s]


199 0.013007371443204957 0.8684117197990417


In [7]:
colors = torch.tensor([[0, 0, 0],         # class 0 'background'  black
                       [255, 0, 0],       # class 1 'grasp'       red
                       [255, 255, 0],     # class 2 'cut'         yellow
                       [0, 255, 0],       # class 3 'scoop'       green
                       [0, 255, 255],     # class 4 'contain'     sky blue
                       [0, 0, 255],       # class 5 'pound'       blue
                       [255, 0, 255],     # class 6 'support'     purple
                       [255, 255, 255]    # class 7 'wrap grasp'  white
                      ])

In [8]:
def class_to_mask(cls):
    
    mask = colors[cls].transpose(1, 2).transpose(1, 3)
    
    return mask

In [30]:
def predict(model, sample, device='cpu'):
    model.eval()
    model.to(device)
    
    x, y = sample['image'], sample['class']
    
    
    x = x.to(device)
    y = y.to(device)

    with torch.no_grad():
        _, y_pred = model(x).max(1)    # y_pred.shape => (N, 240, 320)
    
    true_mask = class_to_mask(y).to('cpu')
    pred_mask = class_to_mask(y_pred).to('cpu')
    
    save_image(true_mask, "./SegNet_without_class_weights_results/true_mask_with_SegNet_without_class_weights.jpg")
    save_image(pred_mask, "./SegNet_without_class_weights_results/pred_mask_with_SegNet_without_class_weights.jpg")

In [59]:
trained_model = SegNetBasic(3, 8)
trained_model.load_state_dict(torch.load("./SegNet_without_class_weights_results/final_model.prm"))

In [32]:
eval_data = PartAffordanceDataset('eval.csv',
                                transform=transforms.Compose([
                                    CenterCrop(),
                                    ToTensor(),
                                    Normalize()
                                ]))

In [33]:
def reverse_normalize(x, mean, std):
    x[:, 0, :, :] = x[:, 0, :, :] * std[0] + mean[0]
    x[:, 1, :, :] = x[:, 1, :, :] * std[1] + mean[1]
    x[:, 2, :, :] = x[:, 2, :, :] * std[2] + mean[2]
    return x

In [36]:
eval_loader = DataLoader(eval_data, batch_size=8, shuffle=False)

mean=[55.8630, 59.9099, 91.7419]
std=[31.6852, 29.8496, 19.0835]

for sample in eval_loader:
    trained_model.eval()
    
    predict(trained_model, sample)
    
    x = sample["image"]
    x = reverse_normalize(x, mean, std)
    save_image(x/255, "./SegNet_without_class_weights_results/original_img_with_SegNet_without_class_weights.jpg")
    
    break

In [6]:
def crop_center_pil_image(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

In [112]:
mean=[55.8630, 59.9099, 91.7419]
std=[31.6852, 29.8496, 19.0835]

def predict_from_image(model, image, device='cpu'):
    model.eval()
    
    image = transforms.functional.resize(image, 420)
    image = crop_center_pil_image(image, 320, 240)
    image = transforms.functional.to_tensor(image)
    image = transforms.functional.normalize(image, mean, std)
    image = image.view((1, 3, 240, 320))
    print(image.shape)
    
    with torch.no_grad():
        _, y_pred = model(image).max(1)
        
    y_pred = class_to_mask(y_pred)
    
    save_image(y_pred, './test_image_with_segnet.jpg')

In [113]:
image = Image.open('Image from iOS.jpg')                                                                                

In [114]:
predict_from_image(trained_model, image)

torch.Size([1, 3, 240, 320])
